# imports

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
from sklearn import preprocessing
from pathlib import Path
from typing import Tuple
import gower
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

# Set display options
pd.set_option('display.max_columns', 100)
pd.set_option("display.max_rows", 2000)

# Set plot style
plt.style.use('fivethirtyeight')

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [2]:
train = pd.read_csv('../Data Files/train_data.csv')
cols = ['date', 'drug_id', 'brand', 'country', 'cluster_nl', 'population', 'public_perc_che', 'che_perc_gdp',
'che_pc_usd', 'insurance_perc_che', 'corporation', 'indication', 'launch_date', 'ind_launch_date',
'therapeutic_area', 'prev_perc', 'price_unit', 'price_month', 'target']
train = train[cols]
print(train.shape)
train.head(5)

(118917, 19)


,date,drug_id,brand,country,cluster_nl,population,public_perc_che,che_perc_gdp,che_pc_usd,insurance_perc_che,corporation,indication,launch_date,ind_launch_date,therapeutic_area,prev_perc,price_unit,price_month,target
0,2014-06-01,DRUG_ID_8795,BRAND_354E,COUNTRY_88A3,BRAND_354E_COUNTRY_88A3,1.008039,1.835821,1.665879,1.209114,1.893333,CORP_D524,['IND_C3B6'],2014-06-01,-1,THER_AREA_980E,0.028367,1.013784,1.006444,1.000784
1,2014-06-01,DRUG_ID_E66E,BRAND_626D,COUNTRY_8B47,BRAND_626D_COUNTRY_8B47,1.023562,-1.000000,-1.000000,-1.000000,-1.000000,CORP_01C7,"['IND_1590', 'IND_ECAC']",2014-06-01,2014-09-01 00:00:00,THER_AREA_96D7,0.000047,1.626677,-1.000000,1.000000
2,2014-06-01,DRUG_ID_F272,BRAND_45D9,COUNTRY_88A3,BRAND_45D9_COUNTRY_88A3,1.008039,1.835821,1.665879,1.209114,1.893333,CORP_39F7,['IND_B2EF'],2014-06-01,-1,THER_AREA_96D7,0.001502,3.144874,-1.000000,1.002258
3,2014-06-01,DRUG_ID_1D4E,BRAND_D724,COUNTRY_445D,BRAND_D724_COUNTRY_445D,1.253186,1.805970,2.051770,1.851280,1.000000,CORP_711A,['IND_BAFB'],2014-06-01,-1,THER_AREA_6CEE,0.001304,1.213446,-1.000000,1.068761
4,2014-06-01,DRUG_ID_AA88,BRAND_4887,COUNTRY_D8B0,BRAND_4887_COUNTRY_D8B0,1.639352,1.880597,2.059130,1.791199,2.013333,CORP_443D,['IND_3F31'],2014-06-01,-1,THER_AREA_6CEE,0.054467,1.008708,1.018589,1.036312


In [3]:
test = pd.read_csv('../Data Files/submission_data.csv')
test = test[cols]
print(test.shape)
test.head(5)

(1769, 19)


,date,drug_id,brand,country,cluster_nl,population,public_perc_che,che_perc_gdp,che_pc_usd,insurance_perc_che,corporation,indication,launch_date,ind_launch_date,therapeutic_area,prev_perc,price_unit,price_month,target
0,2023-01-01,DRUG_ID_FADD,BRAND_E57A,COUNTRY_06E4,BRAND_E57A_COUNTRY_06E4,1.004739,1.910448,1.467920,2.079046,1.000000,CORP_01C7,['IND_C3B6'],2022-02-01,2022-08-01 00:00:00,THER_AREA_980E,0.036647,1.012037,1.007091,NaN
1,2023-01-01,DRUG_ID_E07F,BRAND_014B,COUNTRY_06E4,BRAND_014B_COUNTRY_06E4,1.004739,1.910448,1.467920,2.079046,1.000000,CORP_01C7,['IND_A496'],2022-09-01,-1,THER_AREA_6CEE,0.106925,1.121625,-1.000000,NaN
2,2023-01-01,DRUG_ID_1A21,BRAND_1E6F,COUNTRY_4647,BRAND_1E6F_COUNTRY_4647,3.243906,1.567164,1.108030,1.018268,1.226667,CORP_01C7,"['IND_120F', 'IND_8E8D']",2022-05-01,2022-06-01 00:00:00,THER_AREA_CD59,0.041152,1.005001,-1.000000,NaN
3,2023-01-01,DRUG_ID_1315,BRAND_9259,COUNTRY_FA79,BRAND_9259_COUNTRY_FA79,1.042681,1.786265,1.515375,1.190075,1.785253,CORP_01C7,['IND_BAFB'],2022-12-01,-1,THER_AREA_6CEE,0.002766,1.271214,-1.000000,NaN
4,2023-01-01,DRUG_ID_2832,BRAND_106A,COUNTRY_E319,BRAND_106A_COUNTRY_E319,1.881992,1.223881,1.127864,1.018876,1.076762,CORP_DF1B,['IND_5586'],2022-12-01,2020-03-01 00:00:00,THER_AREA_96D7,0.000133,2.073953,-1.000000,NaN


In [4]:
def split_data_and_check_ratios(train, year=2022):
    """
    Split the data into train and test sets based on the specified year,
    and compute the actual ratio of `drug_id`s with and without historical data.
    
    :param train: The original dataset.
    :param year: The year to use for splitting the test set.
    :return: train (before the specified year), test (the specified year), 
             ratio of `drug_id`s with historical data, ratio of `drug_id`s without historical data.
    """
    print(f"\nOriginal Dataset shape (Before Splitting): {train.shape}")
    # Ensure `date` column is in datetime format
    train["date"] = pd.to_datetime(train["date"])
    
    # Split the data: test = specified year, train = before specified year
    test = train[train["date"].dt.year == year]
    train = train[train["date"].dt.year < year]
    
    # Identify `drug_id`s with historical data (appear in train) and no historical data (do not appear in train)
    drug_ids_with_history = set(train["drug_id"].unique())
    drug_ids_no_history = set(test["drug_id"].unique()) - drug_ids_with_history
    
    # Compute the actual ratio of `drug_id`s with and without historical data
    total_drug_ids = len(drug_ids_with_history) + len(drug_ids_no_history)
    ratio_with_history = len(drug_ids_with_history) / total_drug_ids
    ratio_no_history = len(drug_ids_no_history) / total_drug_ids
    
    # Display the actual ratios
    print(f"\nOriginal Dataset Ratios (Before Splitting):")
    print(f"\nOriginal train Dataset shape (Before Splitting): {train.shape}")
    print(f"\nOriginal test Dataset shape (Before Splitting): {test.shape}")
    print(f"Proportion of `drug_id`s with historical data: {ratio_with_history:.4f}")
    print(f"Proportion of `drug_id`s without historical data: {ratio_no_history:.4f}")
    print(f"drug_ids_with_history:\n {drug_ids_with_history}")
    print(f"drug_ids_no_history:\n {drug_ids_no_history}")
    
    return train, test, ratio_with_history, ratio_no_history

# checking actual splits

In [5]:
# train['date'] = pd.to_datetime(train['date'])

# # Define years to remove
# years_to_remove = [2021]

# # Filter out rows for those years
# train = train[~train['date'].dt.year.isin(years_to_remove)]



# # Verify the unique years in the filtered dataset
# print(train['date'].dt.year.unique())

In [6]:
original = pd.concat([train, test])
original['date'] = pd.to_datetime(original['date'])
yearly_counts = original.groupby(original['date'].dt.year).size()
yearly_counts

date
2014      508
2015     3553
2016     7523
2017    12516
2018    16697
2019    20481
2020    21388
2021    19921
2022    16330
2023     1769
dtype: int64

In [7]:



train, test, ratio_with_history, ratio_no_history = split_data_and_check_ratios(original, year=2023)



Original Dataset shape (Before Splitting): (120686, 19)

Original Dataset Ratios (Before Splitting):

Original train Dataset shape (Before Splitting): (118917, 19)

Original test Dataset shape (Before Splitting): (1769, 19)
Proportion of `drug_id`s with historical data: 0.9885
Proportion of `drug_id`s without historical data: 0.0115
drug_ids_with_history:
 {'DRUG_ID_1D4E', 'DRUG_ID_47B7', 'DRUG_ID_F923', 'DRUG_ID_2757', 'DRUG_ID_C291', 'DRUG_ID_6705', 'DRUG_ID_45AB', 'DRUG_ID_E0F1', 'DRUG_ID_E8D6', 'DRUG_ID_52A5', 'DRUG_ID_98D8', 'DRUG_ID_D5FC', 'DRUG_ID_482D', 'DRUG_ID_83FC', 'DRUG_ID_E73F', 'DRUG_ID_7E51', 'DRUG_ID_FDD8', 'DRUG_ID_032B', 'DRUG_ID_503F', 'DRUG_ID_883A', 'DRUG_ID_7DCA', 'DRUG_ID_B15F', 'DRUG_ID_B4D3', 'DRUG_ID_45DC', 'DRUG_ID_318B', 'DRUG_ID_C7D2', 'DRUG_ID_F784', 'DRUG_ID_FD54', 'DRUG_ID_EE5B', 'DRUG_ID_933E', 'DRUG_ID_3416', 'DRUG_ID_A5ED', 'DRUG_ID_4C1D', 'DRUG_ID_94A5', 'DRUG_ID_CD87', 'DRUG_ID_D450', 'DRUG_ID_6087', 'DRUG_ID_30F8', 'DRUG_ID_A874', 'DRUG_ID_0259',

the original data has 98% data with historical drug_ids and 2% drug_ids with no historical data, and the test set has around 2000 values. We will try to replicate the test set(since we are no longer in the hackathon ) to test modelling and understand how to improve it - this is mainly known as backtesting

# making different test sets

the train data has 118917 rows , we will use year 2022 as the new test set and replicate the original test set.
- In 2022 we see there are 16330 rows, but we need only ~2000 rows with ~98% drug_ids with historical data and ~2% without historical data
- I am just sampling and trying to replicate 8 different datasets ( i used only 3 of them in the end) with the same distribution and shape of orginial test set
- with this I can experiment with our predictions on the replicated test sets and understand how our final metric functions, and it is actually very close to what results we ended up with in the hackathon where are final score was 0.03101447 (18th overall ) , the best results where ~0.0100 (top 5)
- this notebooks aims to achieve ~0.01

In [8]:
def create_balanced_test_set(test, size=2000, history_ratio=0.5, random_state=None):
    """
    Create a balanced test set with a mix of `drug_id`s with and without historical data.
    
    :param test: The test data.
    :param size: Desired size of the test set.
    :param history_ratio: Proportion of `drug_id`s with historical data.
    :param random_state: Seed for reproducibility.
    :return: A balanced test set, list of `drug_id`s with history, list of `drug_id`s without history.
    
    
    """
    print(f'creating test set no {random_state}, with random state {random_state} ')
    
    # Calculate the number of rows for `drug_id`s with and without historical data
    n_history = int(size * history_ratio)
    n_no_history = size - n_history
    
    # Sample `drug_id`s with historical data
    history_drugs = test[test["has_history"]].sample(n=n_history, replace=True, random_state=random_state)
    
    # Sample `drug_id`s without historical data
    no_history_drugs = test[~test["has_history"]].sample(n=n_no_history, replace=True, random_state=random_state)
    
    # Combine the samples into a balanced test set
    balanced_test = pd.concat([history_drugs, no_history_drugs]).sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    # Extract `drug_id`s with and without historical data
    history_drug_ids = history_drugs["drug_id"].unique().tolist()
    no_history_drug_ids = no_history_drugs["drug_id"].unique().tolist()
    
    return balanced_test, history_drug_ids, no_history_drug_ids

# Load your dataset (replace `train` with your actual dataset)
# train = pd.read_csv("your_dataset.csv")

# Split the data and check the actual ratios
train, test, ratio_with_history, ratio_no_history = split_data_and_check_ratios(train, year=2022)

# Add a column to `test` indicating whether each `drug_id` has historical data
test["has_history"] = test["drug_id"].isin(set(train["drug_id"].unique()))

# Create multiple test sets with fixed random seeds for reproducibility
num_test_sets = 8  # Number of test sets to create
test_sets = [create_balanced_test_set(test, size=2000, history_ratio=ratio_with_history, random_state=i) for i in range(num_test_sets)]

# Check the resulting test sets and print `drug_id`s
for i, (test_set, history_drug_ids, no_history_drug_ids) in enumerate(test_sets):
    
    print(f"\nTest Set {i+1} Shape:", test_set.shape)
    print("Proportion of `drug_id`s with historical data:", test_set["has_history"].mean())
    print("`drug_id`s with historical data:", history_drug_ids)
    print("`drug_id`s without historical data:", no_history_drug_ids)


Original Dataset shape (Before Splitting): (118917, 19)

Original Dataset Ratios (Before Splitting):

Original train Dataset shape (Before Splitting): (102587, 19)

Original test Dataset shape (Before Splitting): (16330, 19)
Proportion of `drug_id`s with historical data: 0.9805
Proportion of `drug_id`s without historical data: 0.0195
drug_ids_with_history:
 {'DRUG_ID_1D4E', 'DRUG_ID_47B7', 'DRUG_ID_F923', 'DRUG_ID_2757', 'DRUG_ID_C291', 'DRUG_ID_6705', 'DRUG_ID_45AB', 'DRUG_ID_E0F1', 'DRUG_ID_E8D6', 'DRUG_ID_52A5', 'DRUG_ID_98D8', 'DRUG_ID_D5FC', 'DRUG_ID_482D', 'DRUG_ID_83FC', 'DRUG_ID_E73F', 'DRUG_ID_7E51', 'DRUG_ID_FDD8', 'DRUG_ID_032B', 'DRUG_ID_503F', 'DRUG_ID_883A', 'DRUG_ID_7DCA', 'DRUG_ID_B15F', 'DRUG_ID_B4D3', 'DRUG_ID_45DC', 'DRUG_ID_318B', 'DRUG_ID_C7D2', 'DRUG_ID_F784', 'DRUG_ID_FD54', 'DRUG_ID_EE5B', 'DRUG_ID_933E', 'DRUG_ID_3416', 'DRUG_ID_A5ED', 'DRUG_ID_4C1D', 'DRUG_ID_94A5', 'DRUG_ID_CD87', 'DRUG_ID_D450', 'DRUG_ID_6087', 'DRUG_ID_30F8', 'DRUG_ID_A874', 'DRUG_ID_0259'

In [9]:
test_sets[1][0].head()

,date,drug_id,brand,country,cluster_nl,population,public_perc_che,che_perc_gdp,che_pc_usd,insurance_perc_che,corporation,indication,launch_date,ind_launch_date,therapeutic_area,prev_perc,price_unit,price_month,target,has_history
0,2022-12-01,DRUG_ID_D637,BRAND_0056,COUNTRY_6B71,BRAND_0056_COUNTRY_6B71,1.049628,1.552239,1.684065,1.270827,1.440000,CORP_01C7,['IND_FC21'],2018-01-01,-1,THER_AREA_644A,0.017877,1.021988,1.017883,1.494379,True
1,2022-10-01,DRUG_ID_F19A,BRAND_E8E6,COUNTRY_221C,BRAND_E8E6_COUNTRY_221C,1.664972,1.925373,1.040421,1.056960,1.706667,CORP_3C9A,['IND_A6C2'],2020-10-01,2020-06-01 00:00:00,THER_AREA_6CEE,0.000227,1.003476,1.039887,1.005237,True
2,2022-06-01,DRUG_ID_45AB,BRAND_ED6D,COUNTRY_4253,BRAND_ED6D_COUNTRY_4253,2.737465,1.400022,1.581472,1.104713,1.000000,CORP_A682,['IND_8EA5'],2018-05-01,2019-11-01 00:00:00,THER_AREA_96D7,0.000117,1.199389,1.871923,2.650532,True
3,2022-03-01,DRUG_ID_B050,BRAND_9E22,COUNTRY_D8B0,BRAND_9E22_COUNTRY_D8B0,1.653657,1.907573,2.311103,2.010144,2.000000,CORP_E8B1,['IND_8EA5'],2017-05-01,-1,THER_AREA_96D7,0.018808,1.301499,1.849886,2.074116,True
4,2022-11-01,DRUG_ID_2832,BRAND_106A,COUNTRY_53A5,BRAND_106A_COUNTRY_53A5,1.994766,2.000000,2.007093,1.680087,1.986667,CORP_DF1B,['IND_5586'],2021-11-01,-1,THER_AREA_96D7,0.000110,1.081038,1.710991,2.355909,True


In [10]:
test_sets[1][0][test_sets[1][0]["has_history"] == False]["drug_id"].nunique()

4

In [11]:
test_sets[2][0].head()

,date,drug_id,brand,country,cluster_nl,population,public_perc_che,che_perc_gdp,che_pc_usd,insurance_perc_che,corporation,indication,launch_date,ind_launch_date,therapeutic_area,prev_perc,price_unit,price_month,target,has_history
0,2022-05-01,DRUG_ID_3A6F,BRAND_9FCC,COUNTRY_0309,BRAND_9FCC_COUNTRY_0309,1.497382,1.890506,2.000000,1.759886,2.032511,CORP_01C7,"['IND_DA0B', 'IND_9D10', 'IND_BD8B', 'IND_0883']",2017-08-01,-1,THER_AREA_96D7,0.032827,1.212213,1.129994,1.527548,True
1,2022-11-01,DRUG_ID_45AB,BRAND_76D6,COUNTRY_907E,BRAND_76D6_COUNTRY_907E,1.356278,1.843706,1.918488,1.462828,1.040000,CORP_A682,['IND_8EA5'],2018-01-01,-1,THER_AREA_96D7,0.015903,2.536226,3.064331,9.301589,True
2,2022-08-01,DRUG_ID_F3CA,BRAND_0E5A,COUNTRY_FA79,BRAND_0E5A_COUNTRY_FA79,1.042912,1.794286,1.529843,1.183443,1.785806,CORP_09BB,['IND_F338'],2021-03-01,2021-08-01 00:00:00,THER_AREA_96D7,0.007260,1.282841,1.302948,1.068189,True
3,2022-04-01,DRUG_ID_1B64,BRAND_FE66,COUNTRY_53A5,BRAND_FE66_COUNTRY_53A5,1.994766,2.000000,2.007093,1.680087,1.986667,CORP_39F7,['IND_B2EF'],2020-03-01,-1,THER_AREA_96D7,0.002428,1.195899,1.277728,1.027430,True
4,2022-08-01,DRUG_ID_BE95,BRAND_F148,COUNTRY_8B47,BRAND_F148_COUNTRY_8B47,1.025199,-1.000000,-1.000000,-1.000000,-1.000000,CORP_01C7,['IND_F338'],2019-03-01,2020-03-01 00:00:00,THER_AREA_96D7,0.003552,1.742377,-1.000000,1.142243,True


# Feature engineering

In [12]:

def feature_engineering(train, test):
    """
    Perform feature engineering on the train and test datasets.
    
    :param train: The training dataset.
    :param test: The test dataset.
    :return: targets (for evaluation), data (combined train and test with engineered features).
    """
    # Drop the 'has_history' column from the test set
    test.drop(columns="has_history", inplace=True)
    print("Dropped 'has_history' column from the test set.")

    # Create targets for evaluation
    targets = test[["date", "cluster_nl", "target"]].copy()
    test["target"] = np.nan
    print("Created 'targets' for evaluation.")

    # Drop unnecessary columns
    cols_to_drop = ['che_perc_gdp', 'insurance_perc_che', 'ind_launch_date', 'price_month']
    train.drop(columns=cols_to_drop, inplace=True)
    test.drop(columns=cols_to_drop, inplace=True)
    print(f"Dropped unnecessary columns: {cols_to_drop}.")

    # Handle missing values in 'public_perc_che'
    train.public_perc_che = train.public_perc_che.replace(-1, np.nan)
    test.public_perc_che = test.public_perc_che.replace(-1, np.nan)
    train['public_perc_che'].fillna(train.groupby(["country"])['public_perc_che'].transform('median'), inplace=True)
    test['public_perc_che'].fillna(test.groupby(["country"])['public_perc_che'].transform('median'), inplace=True)
    train.public_perc_che = train.public_perc_che.fillna(train.public_perc_che.median())
    test.public_perc_che = test.public_perc_che.fillna(train.public_perc_che.median())
    print("Handled missing values in 'public_perc_che'.")

    # Handle missing values in 'che_pc_usd'
    train.che_pc_usd = train.che_pc_usd.replace(-1, np.nan)
    test.che_pc_usd = test.che_pc_usd.replace(-1, np.nan)
    train['che_pc_usd'].fillna(train.groupby(["country"])['che_pc_usd'].transform('median'), inplace=True)
    test['che_pc_usd'].fillna(test.groupby(["country"])['che_pc_usd'].transform('median'), inplace=True)
    train.che_pc_usd = train.che_pc_usd.fillna(train.che_pc_usd.median())
    test.che_pc_usd = test.che_pc_usd.fillna(train.che_pc_usd.median())
    print("Handled missing values in 'che_pc_usd'.")

    # Handle missing values in 'price_unit'
    train.price_unit = train.price_unit.replace(-1, np.nan)
    test.price_unit = test.price_unit.replace(-1, np.nan)
    train['price_unit'].fillna(train.groupby(["drug_id"])['price_unit'].transform('median'), inplace=True)
    test['price_unit'].fillna(test.groupby(["drug_id"])['price_unit'].transform('median'), inplace=True)
    print("Handled missing values in 'price_unit'.")

    # Check for remaining missing values
    print("Missing values in train set:")
    print(train.isnull().sum())
    print("Missing values in test set:")
    print(test.isnull().sum())

    # Encode categorical features
    cat_feats = ["indication", "drug_id", "brand", "country", "therapeutic_area", "corporation"]
    for c in cat_feats:
        le = preprocessing.LabelEncoder()
        le.fit(pd.concat((train, test))[c])
        train[c] = le.transform(train[c])
        test[c] = le.transform(test[c])
    print("Encoded categorical features.")

    # Extract date-related features for train set
    train['date'] = pd.to_datetime(train['date'])
    train['launch_date'] = pd.to_datetime(train['launch_date'])
    train['quarter'] = train['date'].dt.quarter.astype("int16")
    train['month'] = train['date'].dt.month.astype("int16")
    train['day'] = train['date'].dt.day.astype("int16")
    train['weekday'] = train['date'].dt.weekday.astype("int16")
    train['quarter_l'] = train['launch_date'].dt.quarter.astype("int16")
    train['month_l'] = train['launch_date'].dt.month.astype("int16")
    train['day_l'] = train['launch_date'].dt.day.astype("int16")
    train['weekday_l'] = train['launch_date'].dt.weekday.astype("int16")
    train['days'] = (train['date'] - train['launch_date']) / np.timedelta64(1, 'D')
    print("Extracted date-related features for train set.")

    # Extract date-related features for test set
    test['date'] = pd.to_datetime(test['date'])
    test['launch_date'] = pd.to_datetime(test['launch_date'])
    test['quarter'] = test['date'].dt.quarter.astype("int16")
    test['month'] = test['date'].dt.month.astype("int16")
    test['day'] = test['date'].dt.day.astype("int16")
    test['weekday'] = test['date'].dt.weekday.astype("int16")
    test['quarter_l'] = test['launch_date'].dt.quarter.astype("int16")
    test['month_l'] = test['launch_date'].dt.month.astype("int16")
    test['day_l'] = test['launch_date'].dt.day.astype("int16")
    test['weekday_l'] = test['launch_date'].dt.weekday.astype("int16")
    test['days'] = (test['date'] - test['launch_date']) / np.timedelta64(1, 'D')
    print("Extracted date-related features for test set.")

    # Drop 'launch_date' column
    train.drop(['launch_date'], axis=1, inplace=True)
    test.drop(['launch_date'], axis=1, inplace=True)
    print("Dropped 'launch_date' column.")

    # Check for drug_ids in test set that are not in train set
    new_drug_ids = list(set(test.drug_id) - set(train.drug_id))
    print(f"Number of new drug_ids in test set: {len(new_drug_ids)}")

    # Add a 'select' column to distinguish train and test sets
    train["select"] = "train"
    test["select"] = "test"

    # Combine train and test sets
    data = pd.concat((train, test))
    print("Combined train and test sets into 'data'.")

    # Sort targets for evaluation
    targets = targets.sort_values(["cluster_nl", "date"])
    print("Sorted 'targets' for evaluation.")

    return targets, data

In [13]:

# train = train[train['date'].dt.year != 2020]
print(train['date'].dt.year.unique())  # Should not include 2020

[2014 2015 2016 2017 2018 2019 2020 2021]


In [14]:
dataset_no=2

# Assuming `train` and `test` are already loaded
targets1, data = feature_engineering(train, test_sets[dataset_no][0])


data.head()
# Save the processed data to a CSV file
data.to_csv(f"../abhi_data/data_dummy_2022_{dataset_no}_final.csv", index=False)
print("Saved processed data to 'data_dummy_2022.csv'.")


Dropped 'has_history' column from the test set.
Created 'targets' for evaluation.
Dropped unnecessary columns: ['che_perc_gdp', 'insurance_perc_che', 'ind_launch_date', 'price_month'].
Handled missing values in 'public_perc_che'.
Handled missing values in 'che_pc_usd'.
Handled missing values in 'price_unit'.
Missing values in train set:
date                0
drug_id             0
brand               0
country             0
cluster_nl          0
population          0
public_perc_che     0
che_pc_usd          0
corporation         0
indication          0
launch_date         0
therapeutic_area    0
prev_perc           0
price_unit          0
target              0
dtype: int64
Missing values in test set:
date                   0
drug_id                0
brand                  0
country                0
cluster_nl             0
population             0
public_perc_che        0
che_pc_usd             0
corporation            0
indication             0
launch_date            0
therapeutic_are

In [15]:
data.tail()

,date,drug_id,brand,country,cluster_nl,population,public_perc_che,che_pc_usd,corporation,indication,therapeutic_area,prev_perc,price_unit,target,quarter,month,day,weekday,quarter_l,month_l,day_l,weekday_l,days,select
1995,2022-05-01,171,19,2,BRAND_076F_COUNTRY_0C7D,1.121505,1.926795,1.127497,59,101,9,0.008770,1.662617,NaN,2,5,1,6,3,7,1,0,1035.0,test
1996,2022-06-01,55,352,33,BRAND_9FCC_COUNTRY_9488,1.276059,1.809115,1.179931,0,220,9,0.043968,1.252813,NaN,2,6,1,2,2,4,1,2,791.0,test
1997,2022-01-01,93,527,32,BRAND_EB6C_COUNTRY_907E,1.356278,1.843706,1.462828,35,30,7,0.015002,1.025082,NaN,1,1,1,5,2,5,1,1,1341.0,test
1998,2022-03-01,143,250,37,BRAND_6FE1_COUNTRY_B6AE,1.040685,1.697551,1.336985,5,152,7,0.128669,1.128001,NaN,1,3,1,1,3,8,1,5,577.0,test
1999,2022-06-01,55,221,36,BRAND_62C7_COUNTRY_AE54,12.767484,1.552239,1.092886,0,222,9,0.007849,1.035044,NaN,2,6,1,2,1,3,1,0,457.0,test


In [16]:
targets1.head()

,date,cluster_nl,target
1980,2022-06-01,BRAND_0056_COUNTRY_06E4,1.112130
320,2022-01-01,BRAND_0056_COUNTRY_0C7D,1.217491
1884,2022-04-01,BRAND_0056_COUNTRY_0C7D,1.209845
1728,2022-10-01,BRAND_0056_COUNTRY_0C7D,1.271516
154,2022-11-01,BRAND_0056_COUNTRY_0C7D,1.311337


In [17]:
targets1.isna().sum()

date          0
cluster_nl    0
target        0
dtype: int64

# modelling

In [18]:
subs = pd.DataFrame(columns=['date', 'cluster_nl', 'target']) 
subs

,date,cluster_nl,target


In [19]:
data = pd.read_csv(f"../abhi_data/data_dummy_2022_{dataset_no}_final.csv")
data = data.reset_index(drop=True)
data.head()

,date,drug_id,brand,country,cluster_nl,population,public_perc_che,che_pc_usd,corporation,indication,therapeutic_area,prev_perc,price_unit,target,quarter,month,day,weekday,quarter_l,month_l,day_l,weekday_l,days,select
0,2014-06-01,120,109,28,BRAND_354E_COUNTRY_88A3,1.008039,1.835821,1.209114,115,193,10,0.028367,1.013784,1.000784,2,6,1,6,2,6,1,6,0.0,train
1,2014-06-01,222,219,30,BRAND_626D_COUNTRY_8B47,1.023562,1.835821,1.472378,0,27,9,0.000047,1.626677,1.000000,2,6,1,6,2,6,1,6,0.0,train
2,2014-06-01,235,151,28,BRAND_45D9_COUNTRY_88A3,1.008039,1.835821,1.209114,27,183,9,0.001502,3.144874,1.002258,2,6,1,6,2,6,1,6,0.0,train
3,2014-06-01,24,477,13,BRAND_D724_COUNTRY_445D,1.253186,1.805970,1.851280,54,185,7,0.001304,1.213446,1.068761,2,6,1,6,2,6,1,6,0.0,train
4,2014-06-01,148,157,43,BRAND_4887_COUNTRY_D8B0,1.639352,1.880597,1.791199,33,41,7,0.054467,1.008708,1.036312,2,6,1,6,2,6,1,6,0.0,train


In [20]:
data.loc[data['select'] == 'train', 'target'] = np.log1p(data.loc[data['select'] == 'train', 'target'])

In [21]:
drug_ids = data[data.select == "test"].drug_id.unique()
print( len(drug_ids) )

195


## forecast recent 

### normal 

In [22]:
def get_forecast(q_ind):

    db = drug_.copy()
    cols_to_drop = ['date', 'corporation', 'cluster_nl','select','target']
    db = db.drop(cols_to_drop, axis = 1)
    db[['therapeutic_area','indication','country','brand']] = db[['therapeutic_area','indication','country','brand']].astype(object)
    db[['quarter','month','day','weekday','quarter_l','month_l','day_l','weekday_l']] = db[['quarter','month','day','weekday','quarter_l','month_l','day_l','weekday_l']].astype(object)
    Dg = gower.gower_matrix(db)

    db = drug_.copy()
    cols_to_drop = ['date','cluster_nl','corporation','select','target']
    tmp = db.drop(cols_to_drop, axis = 1)
    tmp_cols = pd.get_dummies(tmp[['therapeutic_area','indication','country','brand','quarter','month','day','weekday','quarter_l','month_l','day_l','weekday_l']].astype(str))
    cols_to_drop = ['therapeutic_area','indication','country','brand','quarter','month','day','weekday','quarter_l','month_l','day_l','weekday_l']
    tmp = tmp.drop(cols_to_drop, axis = 1)
    db = pd.concat((tmp,tmp_cols), axis=1)
    scaler_ = MinMaxScaler()
    db_n = scaler_.fit_transform(db)
    db_n = pd.DataFrame(data=db_n, columns=db.columns, index=db.index)
    De = squareform( pdist(db_n.values) )
    
    r_num = list( db_n.index).index( q_ind )

    selects = np.concatenate((np.argsort( Dg[r_num,:] )[:20], np.argsort( De[r_num,:] )[:20]))
    selects = np.unique( selects )
    res_inds = db_n.index[ selects ]

    t_values = drug_.loc[res_inds, "target"].values
    out = np.median(t_values[~np.isnan(t_values)])

    return out

In [23]:
def get_forecast_country(q_ind):
    f1 = data["country"] == q.loc[q_ind].country

    db = data[ f1 ].copy()
    db[['therapeutic_area','indication','country','brand']] = db[['therapeutic_area','indication','country','brand']].astype(object)
    db[['quarter','month','day','weekday','quarter_l','month_l','day_l','weekday_l']] = db[['quarter','month','day','weekday','quarter_l','month_l','day_l','weekday_l']].astype(object)

    db1 = db.copy()
    cols_to_drop = ['date', 'corporation', 'cluster_nl','select','target','country']
    db1 = db1.drop(cols_to_drop, axis = 1)
    Dg1 = gower.gower_matrix(db1)

    r_num = list(db.index).index( q_ind )

    selects = np.argsort( Dg1[r_num,:] )[:16]
    res_inds = db.index[ selects ]

    t_values = db.loc[res_inds, "target"].values
    out = np.median(t_values[~np.isnan(t_values)])
    
    return out


# def get_forecast_country(q_ind):
#     # Filter data for the same country
#     country = q.loc[q_ind].country
#     db = data[data["country"] == country].copy()
    
#     # Drop unnecessary columns
#     cols_to_drop = ['date', 'corporation', 'cluster_nl', 'select', 'target', 'country']
#     db1 = db.drop(cols_to_drop, axis=1)
    
#     # Convert categorical columns to dummy variables
#     tmp = db1.copy()
#     tmp_cols = pd.get_dummies(tmp[['therapeutic_area', 'indication', 'brand', 'quarter', 'month', 'day', 'weekday', 'quarter_l', 'month_l', 'day_l', 'weekday_l']].astype(str))
#     cols_to_drop = ['therapeutic_area', 'indication', 'brand', 'quarter', 'month', 'day', 'weekday', 'quarter_l', 'month_l', 'day_l', 'weekday_l']
#     tmp = tmp.drop(cols_to_drop, axis=1)
#     db1 = pd.concat((tmp, tmp_cols), axis=1)
    
#     # Normalize the data
#     scaler_ = MinMaxScaler()
#     db1_n = scaler_.fit_transform(db1)
#     db1_n = pd.DataFrame(data=db1_n, columns=db1.columns, index=db1.index)
    
#     # Fit KNN model with Euclidean distance
#     knn = NearestNeighbors(n_neighbors=4, metric='euclidean')
#     knn.fit(db1_n.values)
    
#     # Find the nearest neighbors for the query point
#     r_num = list(db1_n.index).index(q_ind)
#     distances, indices = knn.kneighbors([db1_n.iloc[r_num].values])
    
#     # Get the target values of the nearest neighbors
#     t_values = db.iloc[indices[0]]["target"].values
#     out = np.mean(t_values[~np.isnan(t_values)])
    
#     return out

## lag

In [24]:
## more lag features
def get_features( ndf, id_ ):
    # Lags
    for l_ in [1, 2,4,6,8,10,12]:
        ndf[str(id_)+"_lag_"+str(l_)] = ndf.groupby("cluster_nl")[id_].shift(l_)
    
    # Rolling means and stds
    for r_ in [2,4,6,8,10,12]:
        ndf[str(id_)+"_rmean_"+str(r_)] = ndf.groupby("cluster_nl")[id_].transform(lambda x : x.rolling(r_, closed = 'left').mean())
        ndf[str(id_)+"_rstd_"+str(r_)] = ndf.groupby("cluster_nl")[id_].transform(lambda x : x.rolling(r_, closed = 'left').std())
    
    # Trends
    ndf[str(id_)+"_sell_trend"] = (ndf[str(id_)+"_lag_1"] - ndf[str(id_)+"_rmean_2"]).astype(np.float16)
    
    # Rolling Max
    ndf[str(id_)+"_rmax_2"] = ndf.groupby("cluster_nl")[id_].transform(lambda x: x.rolling(window=2, closed="left").max()).astype(np.float16)

    # Prices
    ndf["price_mean"] = ndf.groupby("cluster_nl")["price_unit"].transform(lambda x: x.mean())
    ndf["price_max"] = ndf.groupby("cluster_nl")["price_unit"].transform(lambda x: x.max())
    ndf["price_min"] = ndf.groupby("cluster_nl")["price_unit"].transform(lambda x: x.min())
    ndf["price_unit_lag_1"] = ndf.groupby("cluster_nl")["price_unit"].shift(1)
    ndf["price_unit_rmean_3"] = ndf.groupby("cluster_nl")["price_unit"].transform(lambda x : x.rolling(2, closed='left').mean())
    ndf["price_unit_rstd_3"] = ndf.groupby("cluster_nl")["price_unit"].transform(lambda x : x.rolling(2, closed ='left').std())


## modelling pipleine

### abhi pipeline

### normal

In [25]:
print(drug_ids)
for d_index in drug_ids :
    ##### part 1 
    drug_ = data[ (data["drug_id"] == d_index) ]
    drug_.drop(columns=["drug_id"], inplace=True)
    
    print( "drug_id", d_index)
    print( drug_.shape )
    print( drug_["select"].value_counts() )
    # display(drug_.head())
    print("part 1 done")
    ##### part 2     
    keep = []
    ignore = []
    for u in drug_.cluster_nl.unique():
        n1 = len(drug_[ (drug_["cluster_nl"] == u ) & ( drug_["select"] == "train") ])
        n2 = len(drug_[ (drug_["cluster_nl"] == u ) & ( drug_["select"] == "test") ])
        print(u,n1,n2, n1-n2)
        if n1 < 13 :
            ignore.append(u)
        else:
            keep.append(u)

    print( "keep/ignore", len(keep), len(ignore) )
    # display(drug_.head())
    print("part 2 done")

    ##### part 3    
    q = drug_[ drug_["cluster_nl"].isin(ignore) & (drug_['select']=="test")]
    
    if len(q) > 0 :
        targets = []
        for idx in list(q.index):
            
            
            # f = get_forecast_country( idx )
            # if np.isnan(f):
            #     f = get_forecast( idx )
            
            # f_1 = get_forecast_country( idx )
            # f_2 = get_forecast( idx )
            
            # if not np.isnan(f_1) and not np.isnan(f_2):
            #     f = 0.5 * (f_1 + f_2)
            # else:
            #     if not np.isnan(f_1):
            #         f = f_1
            #     else:
            #         f = f_2
            
            
            
            f = get_forecast( idx )
            if np.isnan(f):
                f = get_forecast_country( idx )
            
            
            # drug_samples = drug_[drug_["select"] == "train"]  # Filter to training data
            # if len(drug_samples) >= 20:  # Ensure there are at least 20 samples
            #     f = get_forecast(idx)
            # else:
            #     # If not enough samples, use country-level forecast
            #     f = get_forecast_country(idx)
            
            # # If the forecast is still NaN, fall back to country-level forecast
            # if np.isnan(f):
            #     f = get_forecast_country(idx)
            
            
            targets.append( f )
        q["target"] = targets
        subs = pd.concat( (subs, q[["date","cluster_nl","target"]]) )

    print("ignores done.")
    # display(drug_.head())
    print("part 3 done")

    ##### part 4  
    drug_ = drug_[ drug_["cluster_nl"].isin(keep) ]
    
    if len( drug_ ) == 0:
        print("skip next")
        print("---------------------------------------")
        continue
        
    drug_train = drug_[ ~drug_.target.isnull().values ]
    drug_val = drug_[ drug_.target.isnull().values ]
    print( "data size", drug_train.shape, drug_val.shape )
    # display(drug_train.head())

    print("check", set(drug_val.cluster_nl.values) - set(drug_train.cluster_nl.values) )

    drug_train = drug_train.sort_values(["cluster_nl", "date"])
    drug_val = drug_val.sort_values(["cluster_nl", "date"])

    drug_train_n = drug_train.copy()
    drug_val_n = drug_val.copy()

    drug_val_n_com = pd.concat((drug_train_n, drug_val_n))
    drug_val_n_com = drug_val_n_com.sort_values(["cluster_nl", "date"])

    get_features( drug_val_n_com, "target" )
    print("lags done.")

    ##### part 5  
    drug_train_n = drug_val_n_com[ drug_val_n_com.index.isin( drug_train_n.index ) ]
    drug_val_n = drug_val_n_com[ drug_val_n_com.index.isin( drug_val_n.index ) ]

    drug_train_n.dropna(inplace=True)
    print("after lags", drug_train_n.shape)

    print("check", set(drug_val.cluster_nl.values) - set(drug_train.cluster_nl.values) )

    ##### part 6  
    x_train = drug_train_n.drop(['target','cluster_nl','date','select'], axis = 1)
    y_train = drug_train_n["target"].copy()

    print("train data", x_train.shape, y_train.shape)
    
    #     # Define the MAPE function
    # def mape(y_true, y_pred):
    #     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # # Create a custom scorer for MAPE
    # mape_scorer = make_scorer(mape, greater_is_better=False)

    # # Define the parameter grid
    # grid = dict()
    # grid['n_estimators'] = [100, 500]
    # grid['learning_rate'] = [0.01, 0.1]
    # grid['subsample'] = [0.5, 0.7]
    # grid['max_depth'] = [3, 4]

    # # Check the number of samples
    # n_samples = x_train.shape[0]

    # if n_samples == 1:
    #     # If there's only 1 sample, fit a basic model without GridSearchCV
    #     print("Only 1 sample found. Fitting a basic model...")
    #     model = lgb.LGBMRegressor(verbose=-1, n_estimators=100, learning_rate=0.1, subsample=0.7, max_depth=3)
    #     model.fit(x_train, y_train)
    # else:
    #     # If there are more than 1 samples, use GridSearchCV with CV
    #     print(f"{n_samples} samples found. Using GridSearchCV with CV...")
    #     cv_folds = min(2, n_samples)  # Use cv=2 or cv=3, but ensure it doesn't exceed the number of samples
    #     grid_search = GridSearchCV(
    #         estimator=lgb.LGBMRegressor(verbose=-1),
    #         param_grid=grid,
    #         n_jobs=-1,
    #         cv=cv_folds,
    #         scoring=mape_scorer
    #     )
    #     grid_search.fit(x_train, y_train)
    #     model = grid_search.best_estimator_

    # # Print the model details
    # print("Model trained successfully.")
    
    
    model = lgb.LGBMRegressor(n_estimators=300, verbose = -1)
    
    model.fit(x_train, y_train)

    ##### part 7
    print("iterative fit starts")
    for u in drug_val_n.cluster_nl.unique() :
        print(u)

        tmp_train = drug_train_n[ drug_train_n["cluster_nl"] == u ]
        tmp_val = drug_val_n[ drug_val_n["cluster_nl"] == u ]

        print( len(tmp_train), len(tmp_val) )

        if len(tmp_train) != 0:
            for i in range(len(tmp_val)):

                pred = model.predict( tmp_val.drop(['target','cluster_nl','date','select'], axis = 1).iloc[[i]] )
                tmp_val.iloc[[i], tmp_val.columns.get_loc("target")] = pred

                tmp_com = pd.concat((tmp_train, tmp_val))
                get_features( tmp_com, "target" )

                tmp_val = tmp_com[ tmp_com.index.isin( tmp_val.index ) ]

        else:
            pred = model.predict( tmp_val.drop(['target','cluster_nl','date','select'], axis = 1) )
            tmp_val[ "target" ] = pred

        drug_val_n.loc[tmp_val.index] = tmp_val

    print("fit done.")

    ##### part 8
    subs = pd.concat( (subs, drug_val_n[["date","cluster_nl","target"]]) )

    print("drug done -- > next")
    print("---------------------------------------")
    
print(" -- finished -- ")

[ 55  61 239  22 180 207 191 104 213 174 159 217 118  95  49 167  40  85
  73  79  29 112  58 223  66 123   1  14  64 218   9 179 166  35 242 111
 135 198  23  65 230  93  59  24 175 141 224 156 115  84 109  21  41 101
  89 220  98 134  50  17 110 113  15 126 249  27  78 199 127 169 209 157
 190  51  30 228  13   0  46 206 105 193  99 147  36 192  94  92   5  48
 203  31 251  34  81 243 171  25 165  68 194 128 133  54 153 107  71 219
 143 234 250 196  62  28  56 235 227 186 122  26 255 201  83 161  52  43
  19  88 129  53 232  60 148 117 205 132 236 240 164  44  39 170 162 173
 172 216 183  63 106 150  11 187   8 114 168 176 144  75 100 229 182  42
 160   7  18 177 202 238 152 210 149  76 231   2  16  33  47  57 212 120
 140 245 184  70   4 154 188 138 124  82 178  12 233 237 222]
drug_id 55
(2740, 23)
select
train    2698
test       42
Name: count, dtype: int64
part 1 done
BRAND_62C7_COUNTRY_3AD0 60 0 60
BRAND_62C7_COUNTRY_D8B0 60 0 60
BRAND_62C7_COUNTRY_445D 60 0 60
BRAND_62C7_COUNTR

In [26]:
subs.head()

,date,cluster_nl,target
102631,2022-10-01,BRAND_62C7_COUNTRY_AE54,1.176204
104325,2022-11-01,BRAND_9FCC_COUNTRY_C89B,0.700967
104586,2022-06-01,BRAND_62C7_COUNTRY_AE54,1.015626
103324,2022-12-01,BRAND_62C7_COUNTRY_0C7D,0.784228
103035,2022-06-01,BRAND_62C7_COUNTRY_1007,1.199743


In [27]:
subs.isnull().sum()

date          0
cluster_nl    0
target        0
dtype: int64

In [28]:
subs = subs.rename(columns={"target":"prediction"})
subs.head()

,date,cluster_nl,prediction
102631,2022-10-01,BRAND_62C7_COUNTRY_AE54,1.176204
104325,2022-11-01,BRAND_9FCC_COUNTRY_C89B,0.700967
104586,2022-06-01,BRAND_62C7_COUNTRY_AE54,1.015626
103324,2022-12-01,BRAND_62C7_COUNTRY_0C7D,0.784228
103035,2022-06-01,BRAND_62C7_COUNTRY_1007,1.199743


In [29]:
subs['prediction'] = np.expm1(subs['prediction'])
subs.head()

,date,cluster_nl,prediction
102631,2022-10-01,BRAND_62C7_COUNTRY_AE54,2.242045
104325,2022-11-01,BRAND_9FCC_COUNTRY_C89B,1.015701
104586,2022-06-01,BRAND_62C7_COUNTRY_AE54,1.761092
103324,2022-12-01,BRAND_62C7_COUNTRY_0C7D,1.190715
103035,2022-06-01,BRAND_62C7_COUNTRY_1007,2.319263


In [30]:
subs.to_csv(f"../abhi_data/submission_dummy_abhi2022_{dataset_no}_final.csv", sep=",", index=False)

# evaluation

In [ ]:
subs = pd.read_csv(f"../abhi_data/submission_dummy_abhi2022_{dataset_no}_final.csv")
subs = subs.sort_values(["cluster_nl","date"])
subs.head()

,date,cluster_nl,prediction
150,2022-06-01,BRAND_0056_COUNTRY_06E4,1.108262
151,2022-01-01,BRAND_0056_COUNTRY_0C7D,1.171584
152,2022-04-01,BRAND_0056_COUNTRY_0C7D,1.179824
153,2022-10-01,BRAND_0056_COUNTRY_0C7D,1.171924
154,2022-11-01,BRAND_0056_COUNTRY_0C7D,1.166767


In [32]:
subs["target"] = targets1["target"].values
subs.head()

,date,cluster_nl,prediction,target
150,2022-06-01,BRAND_0056_COUNTRY_06E4,1.108262,1.112130
151,2022-01-01,BRAND_0056_COUNTRY_0C7D,1.171584,1.217491
152,2022-04-01,BRAND_0056_COUNTRY_0C7D,1.179824,1.209845
153,2022-10-01,BRAND_0056_COUNTRY_0C7D,1.171924,1.271516
154,2022-11-01,BRAND_0056_COUNTRY_0C7D,1.166767,1.311337


In [33]:
zl = list( set ( test.cluster_nl.unique() ) - set( train.cluster_nl.unique() ) )
zl[:10]

['BRAND_D72F_COUNTRY_1033',
 'BRAND_7E52_COUNTRY_A67D',
 'BRAND_8F35_COUNTRY_6C16',
 'BRAND_A12A_COUNTRY_C51E',
 'BRAND_740F_COUNTRY_0309',
 'BRAND_E57A_COUNTRY_88A3',
 'BRAND_C352_COUNTRY_88A3',
 'BRAND_C352_COUNTRY_445D',
 'BRAND_60F2_COUNTRY_C89B',
 'BRAND_143F_COUNTRY_6F78']

In [34]:
subs["zero_actuals"] = ~subs.cluster_nl.isin(zl)
subs.head()

,date,cluster_nl,prediction,target,zero_actuals
150,2022-06-01,BRAND_0056_COUNTRY_06E4,1.108262,1.112130,True
151,2022-01-01,BRAND_0056_COUNTRY_0C7D,1.171584,1.217491,True
152,2022-04-01,BRAND_0056_COUNTRY_0C7D,1.179824,1.209845,True
153,2022-10-01,BRAND_0056_COUNTRY_0C7D,1.171924,1.271516,True
154,2022-11-01,BRAND_0056_COUNTRY_0C7D,1.166767,1.311337,True


In [35]:

def _CYME(df: pd.DataFrame) -> float:
    """ Compute the CYME metric, that is 1/2(median(yearly error) + median(monthly error))"""
    
    yearly_agg = df.groupby("cluster_nl")[["target", "prediction"]].sum().reset_index()
    yearly_error = abs((yearly_agg["target"] - yearly_agg["prediction"])/yearly_agg["target"]).median()

    monthly_error = abs((df["target"] - df["prediction"])/df["target"]).median()

    print(f"yearly_error : {yearly_error}, monthly_error: {monthly_error}")

    return 1/2*(yearly_error + monthly_error)


def _metric(df: pd.DataFrame) -> float:
    """Compute metric of submission.

    :param df: Dataframe with target and 'prediction', and identifiers.
    :return: Performance metric
    """
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])

    # Split 0 actuals - rest
    zeros = df[df["zero_actuals"] == 1]
    recent = df[df["zero_actuals"] == 0]
    
    print("zeros")
    print(zeros)
    print("recent")
    print(recent)
    
    # weight for each group
    zeros_weight = len(zeros)/len(df)
    recent_weight = 1 - zeros_weight
    
    print(f"zeros_weight:  {zeros_weight}, recent_weight: {recent_weight}")
    
    # Compute CYME for each group
    part1 = recent_weight*_CYME(recent) # recent ha
    
    part2 = zeros_weight*min(1,_CYME(zeros)) # gheyre recent ha
    
    print(f"part1: {part1}, part2: {part2}")
    
    if not np.isnan( part1 ) and not np.isnan( part1 ):
        out = round(part1 + part2, 8)
    else:
        out = round(part2, 8)
    return out


def compute_metric(submission: pd.DataFrame) -> Tuple[float, float]:
    """Compute metric.

    :param submission: Prediction. Requires columns: ['cluster_nl', 'date', 'target', 'prediction']
    :return: Performance metric.
    """

    submission["date"] = pd.to_datetime(submission["date"])
    submission = submission[['cluster_nl', 'date', 'target', 'prediction', 'zero_actuals']]
    
    return _metric(submission)

# Optionally check performance
print("Performance:", compute_metric(subs))

zeros
                   cluster_nl       date     target  prediction  zero_actuals
150   BRAND_0056_COUNTRY_06E4 2022-06-01   1.112130    1.108262          True
151   BRAND_0056_COUNTRY_0C7D 2022-01-01   1.217491    1.171584          True
152   BRAND_0056_COUNTRY_0C7D 2022-04-01   1.209845    1.179824          True
153   BRAND_0056_COUNTRY_0C7D 2022-10-01   1.271516    1.171924          True
154   BRAND_0056_COUNTRY_0C7D 2022-11-01   1.311337    1.166767          True
155   BRAND_0056_COUNTRY_1007 2022-11-01   3.328773    2.880366          True
156   BRAND_0056_COUNTRY_1007 2022-12-01   3.160442    2.892911          True
157   BRAND_0056_COUNTRY_221C 2022-01-01   1.101831    1.117524          True
158   BRAND_0056_COUNTRY_2E65 2022-08-01   1.017308    1.030306          True
159   BRAND_0056_COUNTRY_2E65 2022-09-01   1.022017    1.019640          True
160   BRAND_0056_COUNTRY_3F0E 2022-01-01   1.221783    1.274343          True
161   BRAND_0056_COUNTRY_4442 2022-05-01   2.274637    2.1

# results

In [36]:
# Performance: 0.02934487
# EXPERIMENTS RESULTS

# ------------BASELINE----------------
# test set 0  - 6 minutes to run
# zeros_weight:  0.9355, recent_weight: 0.0645
# yearly_error : 0.03693342164842836, monthly_error: 0.03442995945710092
# yearly_error : 0.027495405951616664, monthly_error: 0.03058615500892749
# part1: 0.0023014690406533193, part2: 0.02716765013929453
# Performance: 0.02946912

#test set 1  - 6 minutes to run
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.04268486088274496, monthly_error: 0.03824872363441356
# yearly_error : 0.026760744740338675, monthly_error: 0.029631399856907083
# part1: 0.002852908854229838, part2: 0.026208249201569965
# Performance: 0.02906116

#test set 2  - 8 minutes to run
# zeros_weight:  0.94, recent_weight: 0.06000000000000005
# yearly_error : 0.03356011748782471, monthly_error: 0.0349476169409979
# yearly_error : 0.02713112040504676, monthly_error: 0.02876025829323739
# part1: 0.00205523203286468, part2: 0.02626894798819355
# Performance: 0.02832418

#-----------KNN  DRUG + COUNTRY----------------

# test set 0  - 19 minutes to run
# zeros_weight:  0.9355, recent_weight: 0.0645
# yearly_error : 0.09169320173510015, monthly_error: 0.08095651228337708
# yearly_error : 0.027662763535215323, monthly_error: 0.030642987211395873
# part1: 0.005567953277095891, part2: 0.027272514911727386
# Performance: 0.03284047

# test set 1  - 25 minutes to run
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.09136579680679932, monthly_error: 0.0801077890849526
# yearly_error : 0.027048458185710663, monthly_error: 0.029828876318610222
# part1: 0.006044443902684256, part2: 0.02643374121088313
# Performance: 0.03247819

#test set 2  - 18 minutes to run 
# zeros_weight:  0.94, recent_weight: 0.06000000000000005
# yearly_error : 0.09566592861899913, monthly_error: 0.08507456146890648
# yearly_error : 0.02713112040504676, monthly_error: 0.02903959436292604
# part1: 0.005422214702637173, part2: 0.026400235940947214
# Performance: 0.03182245

#-----------ONLY COUNTRY---------------- 
# test set 1  - 25 minutes to run
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.07845991954373274, monthly_error: 0.08290072217408267
# yearly_error : 0.026891157828794556, monthly_error: 0.029775082941053486
# part1: 0.005687962620552994, part2: 0.026335635397786876
# Performance: 0.0320236

#----------- ONLY COUNTRY TOP 4---------------- 
# test set 1  - 21 minutes to run
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.07915806380979566, monthly_error: 0.0568585339297157
# yearly_error : 0.02650190457425059, monthly_error: 0.029111600895537347
# part1: 0.0047945850703177756, part2: 0.025846376667083944
# Performance: 0.03064096

#---------DRUG ID TOP 2/4----------------
# test set 1  - 
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.07732206214645955, monthly_error: 0.04736550953716392
# yearly_error : 0.026199185581792767, monthly_error: 0.029088606263435644
# part1: 0.004395236901847728, part2: 0.025695001260069905
# Performance: 0.03009024

#---------DRUG_ID  KNN WITH LOG TARGET----------------
# test set 1  - 
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.06274796181076821, monthly_error: 0.04580291921099955
# yearly_error : 0.02199014588758101, monthly_error: 0.025727597386198515
# part1: 0.0038264185560173136, part2: 0.022176821186489035
# Performance: 0.02600324

#---------DRUG_ID WITH LOG TARGET GRID SEARCH LIGHTGBM----------------
# test set 1  - 
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.03126266390525447, monthly_error: 0.030690032635825506
# yearly_error : 0.021905372531847083, monthly_error: 0.024818301190838978
# part1: 0.0021838325530730692, part2: 0.021714827362618344
# Performance: 0.02389866

#---------DRUG_ID WITH LOG TARGET GRID SEARCH LIGHTGBM----------------
# test set 1  - 
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.03126266390525447, monthly_error: 0.030690032635825506
# yearly_error : 0.021905372531847083, monthly_error: 0.024818301190838978
# part1: 0.0021838325530730692, part2: 0.021714827362618344
# Performance: 0.02389866

#---------DRUG_ID 20, COUNTRY 15 WITH LOG TARGET NORMAL LIGHTGBM----------------
# test set 1  - 7 MINS
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.03126266390525447, monthly_error: 0.030690032635825506
# yearly_error : 0.02162793188877339, monthly_error: 0.023606944439469948
# part1: 0.0021838325530730692, part2: 0.021022908773551095
# Performance: 0.02320674

#---------DRUG_ID 5, COUNTRY KNN 13 WITH LOG TARGET NORMAL LIGHTGBM----------------
# test set 1  - 2 MINS
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.038231430812556866, monthly_error: 0.035628223427350524
# yearly_error : 0.021694837899392846, monthly_error: 0.023535560386310755
# part1: 0.002603552811956736, part2: 0.02102082760328075
# Performance: 0.02362438

#---------DRUG_ID 20, COUNTRY 15 WITH LOG TARGET NORMAL LIGHTGBM----------------
# test set 1  - 7 MINS
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.03126266390525447, monthly_error: 0.030690032635825506
# yearly_error : 0.02162793188877339, monthly_error: 0.023606944439469948
# part1: 0.0021838325530730692, part2: 0.021022908773551095
# Performance: 0.02320674


#---------DRUG_ID 20, COUNTRY 15 WITH LOG TARGET NORMAL LIGHTGBM----------------
# test set 0  - 7 MINS
# zeros_weight:  0.9355, recent_weight: 0.0645
# yearly_error : 0.02489168386193652, monthly_error: 0.02585101835474924
# yearly_error : 0.021672824007509073, monthly_error: 0.024304988849832124
# part1: 0.0016364521464881157, part2: 0.021506121964021345
# Performance: 0.02314257


#---------DRUG_ID 20, COUNTRY 15 WITH LOG TARGET NORMAL LIGHTGBM----------------
# test set 1  - 7 MINS
# zeros_weight:  0.9295, recent_weight: 0.07050000000000001
# yearly_error : 0.03126266390525447, monthly_error: 0.030690032635825506
# yearly_error : 0.02162793188877339, monthly_error: 0.023606944439469948
# part1: 0.0021838325530730692, part2: 0.021022908773551095
# Performance: 0.02320674

#---------DRUG_ID 20, COUNTRY 15 WITH LOG TARGET NORMAL LIGHTGBM----------------
# test set 2  - 7 MINS
# zeros_weight:  0.94, recent_weight: 0.06000000000000005
# yearly_error : 0.022360337490383506, monthly_error: 0.02554593268138143
# yearly_error : 0.021643040149513418, monthly_error: 0.024137398278783848
# part1: 0.0014371881051529493, part2: 0.021516806061299715
# Performance: 0.02295399

In [37]:
# check if knn data and dummy data are same
# make groups for each historical data and increase the lags respectively for each of them groups
# run models and hypertune them?
# run knn or cosine similarity for the data

In [38]:
#changes made 
#knn country 4 and drug 5 - works
#made the knn with median aggregation - works
#log transform target - works good
# outlier removal - useless
#remove 2020 - useless
#made country 15 and drug 20